In [1]:
import numpy as np
import xarray as xr
import re
import matplotlib.pyplot as plt
import imageio
import os

In [31]:
# Cargar el dataset NetCDF
data_path = "/home/gacuervol/Doctorado/DB/GraphCast_data/IBI_SST_L4_44d.nc"
ds = xr.open_dataset(data_path)
ds

<xarray.Dataset>
Dimensions:         (time: 44, latitude: 300, longitude: 300)
Coordinates:
  * latitude        (latitude) float64 19.57 19.62 19.68 ... 34.42 34.48 34.52
  * longitude       (longitude) float64 -20.93 -20.88 -20.82 ... -6.025 -5.975
  * time            (time) datetime64[ns] 1982-01-01 1982-01-02 ... 1982-02-13
Data variables:
    analysed_sst    (time, latitude, longitude) float32 ...
    analysis_error  (time, latitude, longitude) float32 ...
Attributes: (12/34)
    Conventions:                CF-1.4
    cdm_data_type:              grid
    cmems_product_id:           cmems_SST_ATL_SST_L4_REP_OBSERVATIONS_010_026
    contact:                    emmanuelle.autret@ifremer.fr,jean.francois.pi...
    date_created:               20201113T102920Z
    easternmost_longitude:      12.98
    ...                         ...
    stop_time:                  19820101T120000Z
    summary:                    Sea Surface Temperature Analysis from the Eur...
    temporal_resolution:        daily
    title:                      European North West Shelf/Iberia Biscay Irish...
    uuid:                       
    westernmost_longitude:      -20.98

In [ ]:
# Cargar el dataset NetCDF
data_path = "/home/gacuervol/Doctorado/DB/GraphCast_data/IBI_SST_L4_FULL.nc"
ds = xr.open_dataset(data_path).isel(time=slice(-44, None))
ds

<xarray.Dataset>
Dimensions:         (time: 44, latitude: 300, longitude: 300)
Coordinates:
  * latitude        (latitude) float64 19.57 19.62 19.68 ... 34.42 34.48 34.52
  * longitude       (longitude) float64 -20.93 -20.88 -20.82 ... -6.025 -5.975
  * time            (time) datetime64[ns] 2020-11-18 2020-11-19 ... 2020-12-31
Data variables:
    analysed_sst    (time, latitude, longitude) float32 ...
    analysis_error  (time, latitude, longitude) float32 ...
Attributes: (12/34)
    Conventions:                CF-1.4
    cdm_data_type:              grid
    cmems_product_id:           cmems_SST_ATL_SST_L4_REP_OBSERVATIONS_010_026
    contact:                    emmanuelle.autret@ifremer.fr,jean.francois.pi...
    date_created:               20201113T102920Z
    easternmost_longitude:      12.98
    ...                         ...
    stop_time:                  19820101T120000Z
    summary:                    Sea Surface Temperature Analysis from the Eur...
    temporal_resolution:        daily
    title:                      European North West Shelf/Iberia Biscay Irish...
    uuid:                       
    westernmost_longitude:      -20.98

In [51]:
# Cargar el dataset NetCDF
data_path = "/home/gacuervol/Doctorado/DB/GraphCast_data/IBI_SST_L4_FULL.nc"
centtral_day = 13800
ds = xr.open_dataset(data_path).isel(time=slice(centtral_day-22, centtral_day+22))
ds

<xarray.Dataset>
Dimensions:         (time: 44, latitude: 300, longitude: 300)
Coordinates:
  * latitude        (latitude) float64 19.57 19.62 19.68 ... 34.42 34.48 34.52
  * longitude       (longitude) float64 -20.93 -20.88 -20.82 ... -6.025 -5.975
  * time            (time) datetime64[ns] 2019-09-22 2019-09-23 ... 2019-11-04
Data variables:
    analysed_sst    (time, latitude, longitude) float32 ...
    analysis_error  (time, latitude, longitude) float32 ...
Attributes: (12/34)
    Conventions:                CF-1.4
    cdm_data_type:              grid
    cmems_product_id:           cmems_SST_ATL_SST_L4_REP_OBSERVATIONS_010_026
    contact:                    emmanuelle.autret@ifremer.fr,jean.francois.pi...
    date_created:               20201113T102920Z
    easternmost_longitude:      12.98
    ...                         ...
    stop_time:                  19820101T120000Z
    summary:                    Sea Surface Temperature Analysis from the Eur...
    temporal_resolution:        daily
    title:                      European North West Shelf/Iberia Biscay Irish...
    uuid:                       
    westernmost_longitude:      -20.98

In [52]:
def get_coords_keys(da: xr.Dataset) -> tuple:
    """
    Get the latitude and longitude coordinate keys from a da dataset.
    
    Parameters:
    da (xarray.Dataset): The dataset containing da data.
    
    Returns:
    tuple: A tuple containing the latitude and longitude coordinate keys.
    """
    # Find latitude and longitude keys using regex
    lat_key = [
        key for key in da.dims 
        if re.compile(r'^lat', re.IGNORECASE).match(key)
        ][0]
    lon_key = [
        key for key in da.dims 
        if re.compile(r'^lon', re.IGNORECASE).match(key)
        ][0]
    return lat_key, lon_key

In [53]:
def nc_frame_to_png(ax: plt.Axes, da: xr.DataArray, i: int, frame_dir: str):
    """
    Create a PNG file from a 2D xarray DataArray representing sea surface temperature (SST)
    for a specific day, and save it to the specified directory.
    """
    assert isinstance(da, xr.DataArray), "El argumento da debe ser un DataArray de xarray."

     # Get the spatial coordinates
    lat_key, lon_key = get_coords_keys(da)
    lons = da.coords[lon_key].values
    lats = da.coords[lat_key].values

    # Create custom colormap with transparent NaNs
    cmap = plt.cm.coolwarm.copy()
    cmap.set_bad(color=(0, 0, 0, 0))  # Transparent for NaNs

    # Mask the NaNs so imshow respects them
    data = da.isel(time=i).data
    masked_data = np.ma.masked_invalid(data)

    # Clear the axes
    ax.clear()
    ax.axis('off')

    im = ax.imshow(
        masked_data,
        cmap=cmap,
        vmin=np.nanmin(data),
        vmax=np.nanmax(data),
        origin='lower'
    )

    # Save the figure with transparent background and no padding
    ax.figure.savefig(
        f"{frame_dir}/frame_{i:03d}.png",
        transparent=True,
        bbox_inches='tight',
        pad_inches=0
    )
    plt.close(ax.figure)

In [54]:
# Crear carpeta para los frames
frame_dir = '/home/gacuervol/kml_ani'
# 
fig, ax = plt.subplots(figsize=(8, 6))

# Crear imágenes PNG por cada timestep
for i, _ in enumerate(ds.time):
    frame = i
    nc_frame_to_png(ax, ds.analysed_sst, frame, frame_dir)

In [55]:
# Crear el GIF con imageio
from io import BytesIO
import imageio.v2 as imageio
lat_key, lon_key = get_coords_keys(da)
lons = da.coords[lon_key].values
lats = da.coords[lat_key].values
da = ds.analysed_sst

frames = []

for i, t in enumerate(ds.time):
    fig, ax = plt.subplots(figsize=(8, 6))
    da_day = da.isel(time=i)

    im = ax.pcolormesh(lons, lats, da_day, shading='auto', cmap='coolwarm',
                       vmin=da.min().item(), vmax=da.max().item())
    
    ax.set_title(f'SST: {np.datetime_as_string(t.values, unit="D")}')
    plt.colorbar(im, ax=ax, label='SST (°C)')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')

    # Guardar la imagen en memoria (no en disco)
    buf = BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    frames.append(imageio.imread(buf))
    buf.close()
    plt.close()

# Guardar GIF final
imageio.mimsave(f"{frame_dir}/sst_animation.gif", frames, fps=5)

In [56]:
from datetime import datetime, timedelta

kml_path = f"{frame_dir}/animacion.kml"

kml_header = """<?xml version="1.0" encoding="UTF-8"?>
<kml xmlns="http://www.opengis.net/kml/2.2"
     xmlns:gx="http://www.google.com/kml/ext/2.2">
<Document>
"""
kml_footer = "</Document></kml>"


kml_content = kml_header

frame_duration = timedelta(days=1)  # duración por imagen
frames = [f for f in sorted(os.listdir(frame_dir)) if f.endswith('.png')]
# Coordenadas mínimas y máximas
da = ds.analysed_sst
# Get the start time from the first time coordinate
dt64 = da.time.data[0]
dt = dt64.astype('M8[ms]').astype(datetime)
start_time = datetime(
    dt.year, dt.month, dt.day, 
    dt.hour, dt.minute, dt.second
    )
lat_key, lon_key = get_coords_keys(da)
north = float(da.coords[lat_key].max())
south = float(da.coords[lat_key].min())
east = float(da.coords[lon_key].max())
west = float(da.coords[lon_key].min())

for i, frame in enumerate(frames):
    timestamp_start = start_time + i * frame_duration
    timestamp_end = timestamp_start + frame_duration
    kml_content += f"""
<GroundOverlay>
    <TimeSpan>
        <begin>{timestamp_start.isoformat()}Z</begin>
        <end>{timestamp_end.isoformat()}Z</end>
    </TimeSpan>
    <Icon>
        <href>{frame}</href>
    </Icon>
    <LatLonBox>
        <north>{north}</north>
        <south>{south}</south>
        <east>{east}</east>
        <west>{west}</west>
    </LatLonBox>
</GroundOverlay>
"""

kml_content += kml_footer

with open(kml_path, "w") as f:
    f.write(kml_content)